In [1]:
!pip install torch

In [2]:
!pip install razdel

In [3]:
!pip install torchtext

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

from razdel import tokenize
from razdel import sentenize
import numpy as np

import random
import math
import time
import json

In [5]:
def text2sent(text):
    return list(sentenize(text))

def text2token(text):
    return list(tokenize(text))

In [6]:
with open('two_way.json', 'r', encoding='utf8') as rp:
    a = json.load(rp)
    

In [7]:
a[0]['article text'][2]

{'entailment score': 0.6250573396682739,
 'comment': 'Поэтому после признания в 2019 году предприятия банкротом, АО продолжило поставки.',
 'text': 'Указанное понимание не соотносится и с буквальным смыслом абзаца второго пункта 1 статьи 134 Закона о банкротстве, закрепляющего единственный критерий внеочередного удовлетворения требований кредитора - направленность платежей на предотвращение катастроф либо гибели людей, вне зависимости от того, связано ли обязательство по внесению платежей с исполнением обязанностей по договору, заключенному в рамках обычной хозяйственной деятельности должника, или нет.'}

In [8]:
a[10]['article text'][2]

{'entailment score': 1.86528742313385,
 'comment': 'Документ, в частности, регламентирует основные процессы по производству, хранению, реализации, транспортировке и использованию семян растений, а также по их импорту и экспорту из России.',
 'text': 'Настоящий Федеральный закон регулирует отношения в области семеноводства (производства (выращивания), хранения, транспортировки, реализации семян сельскохозяйственных растений, их использования), включая оказание услуг в указанной области, а также ввоз семян сельскохозяйственных растений в Российскую Федерацию и вывоз семян из Российской Федерации.'}

In [9]:
X_data = []
y_data = []

for s in range(len(a[0]['article text'])):
    X_data.append(a[0]['article text'][s]['comment'])
    y_data.append(a[0]['article text'][s]['text'])

In [10]:
X_data

['В том числе компания поставляет тепло и горячую воду предприятию, эксплуатирующему шахту "Интинская".',
 'Шахта считается опасным объектом.',
 'Поэтому после признания в 2019 году предприятия банкротом, АО продолжило поставки.',
 'Чтобы избежать возможного ЧП, эту теплоэнергию банкрот частично оплатил в приоритетном порядке.',
 'Но это вызвало протест налоговой службы.',
 'И она обратилась в суд.',
 'Арбитражные суды признали платежи недействительными сделками.',
 'А ответственность за возможные последствия возложили на самого кредитора.',
 'По мнению АО, такое толкование нормы "не обеспечивает защиты прав и интересов ресурсоснабжающей организации".',
 'Вот позиция КС.',
 '"Организация, находящаяся в процессе банкротства, и поставщик энергоресурса должны четко понимать, когда их отношения подпадают под действие соответствующей нормы.',
 'Специфика опасных производств и условия их взаимодействия предполагают наличие специальных технических знаний, а их недостаточность должна восполнят

In [11]:
y_data

['Указанное понимание не соотносится и с буквальным смыслом абзаца второго пункта 1 статьи 134 Закона о банкротстве, закрепляющего единственный критерий внеочередного удовлетворения требований кредитора - направленность платежей на предотвращение катастроф либо гибели людей, вне зависимости от того, связано ли обязательство по внесению платежей с исполнением обязанностей по договору, заключенному в рамках обычной хозяйственной деятельности должника, или нет.',
 '3. Абзац второй пункта 1 статьи 134 Федерального закона "О несостоятельности (банкротстве)", признанный настоящим Постановлением не соответствующим Конституции Российской Федерации, подлежит применению - до внесения в действующее правовое регулирование изменений, вытекающих из настоящего Постановления, - с учетом следующего.',
 'Указанное понимание не соотносится и с буквальным смыслом абзаца второго пункта 1 статьи 134 Закона о банкротстве, закрепляющего единственный критерий внеочередного удовлетворения требований кредитора -

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [14]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [16]:
INPUT_DIM = 2
OUTPUT_DIM = 2
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [17]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(2, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=2, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,358,466 trainable parameters


In [19]:
optimizer = optim.Adam(model.parameters())

In [21]:
from torchtext.data import Field, BucketIterator

ImportError: cannot import name 'Field' from 'torchtext.data' (C:\Users\Марк\AppData\Roaming\Python\Python38\site-packages\torchtext\data\__init__.py)

In [37]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

NameError: name 'Field' is not defined

In [36]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

NameError: name 'TRG' is not defined